In [ ]:
%load_ext autoreload
%autoreload 2
import matplotlib
from tools import *
import pandas as pd
from os.path import join as path_join
import numpy as np
import math
from matplotlib import pyplot as plt
from tqdm.notebook import tqdm as tqdm
import matplotlib
import os
from matplotlib import cm
from matplotlib.colors import ListedColormap

In [ ]:
def plt_export(name):
    global STUDY_PARAM
    plt.savefig(f'./data/figs/{name}.pdf', bbox_inches='tight')


In [ ]:
TEMP_DIR = f"./data/tmp_pickles/"
DATA_DIR = f"./data/"

def pickelize_to_TEMP_DIR(saving_obj, file_name):
    pickelize(saving_obj, path_join(TEMP_DIR, file_name))


def un_pickelize_from_TEMP_DIR(file_name):
    return un_pickelize(path_join(TEMP_DIR, file_name))


BUCKETS_NO = 4
MIN_INTERACTIONS = 10

In [ ]:
def get_raw_plt():
    import matplotlib.pyplot as plt
    plt.rcParams.update(matplotlib.rcParamsDefault)
    plt.rcParams.update({
        "font.family": "DejaVu Sans",  # use serif/main font for text elements
        "font.size": "18",
        "figure.figsize": (20, 10),
        # "font.weight": "bold",
    })
    return plt

In [ ]:
articles_df = pd.read_csv(path_join(DATA_DIR, 'articles.csv'))
all_included_urls = list(articles_df['canonical_url'])

In [ ]:
articles_df

In [ ]:
if not os.path.exists(path_join(TEMP_DIR, "all_posts_info_0.pkl")):
    all_posts_info = {}
    for url in tqdm(all_included_urls):
        all_posts_info[url] = get_all_posts_of_article(url)
    pickelize_to_TEMP_DIR(all_posts_info, "all_posts_info_0.pkl")
else:
    all_posts_info = un_pickelize_from_TEMP_DIR("all_posts_info_0.pkl")

In [ ]:
if os.path.exists(path_join(TEMP_DIR, "all_posts_info_1.pkl")):
    all_posts_info = un_pickelize_from_TEMP_DIR("all_posts_info_1.pkl")
else:
    for url in tqdm(all_included_urls):
        update_posts_time_series_to_seconds(all_posts_info[url])
    pickelize_to_TEMP_DIR(all_posts_info, "all_posts_info_1.pkl")

In [ ]:
cnt = sum(1 for url in all_posts_info if len(all_posts_info[url]) > 0)
cnt

In [ ]:
import os

if os.path.exists(path_join(TEMP_DIR, "all_posts_info_2.pkl")):
    all_posts_info = un_pickelize_from_TEMP_DIR("all_posts_info_2.pkl")
else:
    stats = {
        "total_number_of_posts": 0,
        "dropped_posts_counter_not_having_history": 0,
        "dropped_posts_counter_not_having_enough_interaction": 0,
        "sum_interaction_included": 0,
        "sum_not_included": 0,
    }

    def process_article(posts):
        new_posts = []
        for post in posts:
            stats["total_number_of_posts"] += 1
            post_history = post["history"]

            if len(post_history["all_interactions"]) > 0:
                if post_history["all_interactions"][-1] >= 10:
                    stats["sum_interaction_included"] += post_history["all_interactions"][-1]
                    new_posts.append(post)
                else:
                    stats["sum_not_included"] += post_history["all_interactions"][-1]
                    stats["dropped_posts_counter_not_having_enough_interaction"] += 1
            else:
                stats["dropped_posts_counter_not_having_history"] += 1

        return new_posts

    all_posts_info = {article_index: process_article(posts) for article_index, posts in all_posts_info.items()}

    print(f"total_number_of_posts: {stats['total_number_of_posts']}")
    print(f"dropped_posts_counter_not_having_history: {stats['dropped_posts_counter_not_having_history']}")
    print(f"dropped_posts_counter_not_having_enough_interaction: {stats['dropped_posts_counter_not_having_enough_interaction']}")
    print(f"remaining number of posts: {stats['total_number_of_posts'] - stats['dropped_posts_counter_not_having_history'] - stats['dropped_posts_counter_not_having_enough_interaction']}")
    print(f"sum_interaction_included: {stats['sum_interaction_included']}")
    print(f"sum_not_included: {stats['sum_not_included']}")

    pickelize_to_TEMP_DIR(all_posts_info, "all_posts_info_2.pkl")


In [ ]:
if os.path.exists(path_join(TEMP_DIR, "all_posts_info_3.pkl")):
    all_posts_info = un_pickelize_from_TEMP_DIR("all_posts_info_3.pkl")
else:
    new_all_posts_info = {}
    for article_index in all_posts_info.keys():
        if len(all_posts_info[article_index]) > 0:
            new_all_posts_info[article_index] = all_posts_info[article_index]
    print(len(all_posts_info), len(new_all_posts_info))
    pickelize_to_TEMP_DIR(new_all_posts_info, "all_posts_info_3.pkl")

In [ ]:
len(new_all_posts_info.keys())

In [ ]:
all_included_urls = list(all_posts_info.keys())
len(all_included_urls)

In [ ]:
count = 0
for url in all_posts_info.keys():
    t = all_posts_info[url]
    for post in t:
        count += max(post["history"]["all_interactions"])
count

In [ ]:
len(all_posts_info)

In [ ]:
import itertools
import operator


def seconds_converter(time):
    day = time // (24 * 3600)
    time = time % (24 * 3600)
    hour = time // 3600
    time %= 3600
    minutes = time // 60
    time %= 60
    seconds = time
    return ("{}d:{}h:{}m:{}s".format(day, hour, minutes, seconds))


In [ ]:
excluded_posts = set()
excluded_posts = un_pickelize_from_TEMP_DIR("excluded_posts")##### comment in the first run 

In [ ]:
max_date_in_seconds = 0
for index in all_posts_info.keys():
    for post in all_posts_info[index]:
        post_history = post["history"]
        if max(post_history["date"]) > max_date_in_seconds:
            max_date_in_seconds = max(post_history["date"])
print(seconds_converter(max_date_in_seconds))


In [ ]:
seconds_converter(max_date_in_seconds)

In [ ]:
from tqdm import tqdm

def get_cumulative_time_series_exponential_version(seed, factor):
    global excluded_posts
    bucket_centers = []
    bc = seed
    while bc <= max_date_in_seconds:
        bucket_centers.append(bc)
        bc *= factor
    bucket_centers.append(max_date_in_seconds)

    yhat = []
    xhat = []

    for i in range(len(bucket_centers)):
        all_max_values = [
            np.max(post["history"]["all_interactions"][post["history"]["date"] <= bucket_centers[i]]) * 100 / np.max(post["history"]["all_interactions"])
            for url in all_posts_info.keys()
            for post in all_posts_info[url]
            if post["platformId"] not in excluded_posts
            and len(post["history"]["all_interactions"][post["history"]["date"] <= bucket_centers[i]]) > 0
        ]

        xhat.append(bucket_centers[i])
        yhat.append(np.mean(all_max_values))
        print(xhat[-1], yhat[-1])

    return xhat, yhat


In [ ]:
x_new, y_new = get_cumulative_time_series_exponential_version(900, 1.2)

In [ ]:
plt = get_raw_plt()
plt.plot(x_new, y_new)
plt.xscale("log")

y_points = []
for i in range(0, 99, 100//BUCKETS_NO):
    y_points.append(i)
y_points.append(99)

y_points_thresh = []
x_points_thresh = []
ii = []
for v in y_points[1:]:
    for i in range(len(y_new)):
        if y_new[i] >= v:
            ii.append(i)
            m = (y_new[i]-y_new[i-1])/(x_new[i]-x_new[i-1])
            x_points_thresh.append(int(x_new[i-1] + (v-y_new[i-1])/m))
            y_points_thresh.append(v)
            break
for i in range(len(y_points_thresh)):
    plt.vlines(x=x_points_thresh[i], ymin=0,
               ymax=y_points_thresh[i], colors='r', linestyles='dashed')
    plt.hlines(y=y_points_thresh[i], xmin=0,
               xmax=x_points_thresh[i], colors='r', linestyles='dashed')

x_points_thresh.append(x_new[-1])

x_labels = [seconds_converter(x_points_thresh[i])
            for i in range(len(x_points_thresh))]

plt.xscale("log")
plt.ylim(0, max(y_new))
plt.xlim(0, x_points_thresh[-1])
plt.xticks(ticks=x_points_thresh, labels=x_labels, rotation=45, ha="right")
plt.yticks([int(temp) for temp in y_points_thresh], [int(temp)
           for temp in y_points_thresh])
# plt.savefig("InteractionsFraction.pdf",bbox_inches='tight')

plt.xlabel("Time after posting (log scale)")
plt.ylabel("Interactions fraction (percentage)", labelpad=10)
plt.legend()
plt.show()

In [ ]:
def get_raw_plt1():
    import matplotlib.pyplot as plt
    plt.rcParams.update(matplotlib.rcParamsDefault)
    plt.rcParams.update({
        # "font.family": "Times",  # use serif/main font for text elements
        "font.size": "30",
        "figure.figsize": (20, 10),
        # "font.weight": "bold",

    })
    return plt


plt = get_raw_plt1()
plt.plot(x_new, y_new)
plt.xscale("log")

y_points = []
for i in range(0, 99, 100//BUCKETS_NO):
    y_points.append(i)
y_points.append(99)

y_points_thresh = []
x_points_thresh = []
ii = []
for v in y_points[1:]:
    for i in range(len(y_new)):
        if y_new[i] >= v:
            ii.append(i)
            m = (y_new[i]-y_new[i-1])/(x_new[i]-x_new[i-1])
            x_points_thresh.append(int(x_new[i-1] + (v-y_new[i-1])/m))
            y_points_thresh.append(v)
            break
for i in range(len(y_points_thresh)):
    plt.vlines(x=x_points_thresh[i], ymin=0,
               ymax=y_points_thresh[i], colors='r', linestyles='dashed')
    plt.hlines(y=y_points_thresh[i], xmin=0,
               xmax=x_points_thresh[i], colors='r', linestyles='dashed')

x_points_thresh.append(x_new[-1])

x_labels = ['1 h, 17 min',
            '5 h, 16 min',
            '17 h, 28 min',
            '5 d, 4 h, 45 min',
            '22 d']

plt.xscale("log")
plt.ylim(0, max(y_new))
plt.xlim(0, x_points_thresh[-1])
plt.xticks(ticks=x_points_thresh, labels=x_labels, rotation=30, ha="right")
plt.yticks([int(temp) for temp in y_points_thresh], [int(temp)
           for temp in y_points_thresh])


plt.xlabel("Time after posting (log scale)")
plt.ylabel("Interactions fraction (percentage)", labelpad=10)
plt_export("InteractionsFraction")

plt.show()


In [ ]:
x_points_thresh

In [ ]:
buckets_thresholds = x_points_thresh[:-2]
buckets_thresholds.append(max_date_in_seconds)
buckets_thresholds.insert(0, 0)

In [ ]:
buckets_thresholds

In [ ]:
for elem in buckets_thresholds:
    print(seconds_converter(elem))

In [ ]:
def get_related_bucket(time):
    for i in range(len(buckets_thresholds)-1):
        if time >= buckets_thresholds[i] and time < buckets_thresholds[i+1]:
            return i
    raise Exception("time is out of range")


In [ ]:
previous_excluded_posts = excluded_posts

def should_exclude_post(post, buckets_thresholds):
    post_history = post["history"]
    for i in range(len(buckets_thresholds) - 1):
        thres1 = buckets_thresholds[i]
        thres2 = buckets_thresholds[i + 1]
        t = (post_history["date"] < thres2) & (post_history["date"] >= thres1)
        post_values = post_history["all_interactions"][t]
        if post_values.shape[0] == 0:
            return True
    return False

excluded_posts = {
    post["platformId"]
    for url in all_posts_info.keys()
    for post in all_posts_info[url]
    if should_exclude_post(post, buckets_thresholds)
}

if previous_excluded_posts == excluded_posts:
    print("excluding posts has converged")
else:
    print("excluding posts has not converged")


In [ ]:
pickelize_to_TEMP_DIR(excluded_posts, "excluded_posts")

In [ ]:
len(excluded_posts)

In [ ]:
# if the excluded posts has not converged run the above procedure again with uncommented excluded_posts = un_pickelize_from_TEMP_DIR("excluded_posts")

In [ ]:
len(all_posts_info.keys())

In [ ]:
articles_df = articles_df[articles_df["canonical_url"].isin(all_posts_info.keys())]

In [ ]:
articles_df.describe()

In [ ]:
articles_df.shape

In [ ]:
articles_df["source"].unique().shape

In [ ]:
len(all_included_urls)

In [ ]:
reliability_thresholds = [0+i*16 for i in range(5)]
bias_thresholds = [-42, -18, -6, 6, 18, 42]
reliability_thresholds, bias_thresholds, buckets_thresholds

bias_classes = [
    "Far left",
    "Left",
    "Balanced",
    "Right",
    "Far right",
    "All"]


reliability_classes = [
    "Most unreliable",
    "Unreliable",
    "Reliable",
    "Most reliable",
    "All"]


In [ ]:
count = 0 
for j in tqdm(range(len(bias_thresholds)-1)):
    if j == len(bias_thresholds)-1:
        bias_indexes = articles_df.index
    elif j == 0 or j == 4:
        bias_indexes = articles_df[(articles_df["bias"] >= bias_thresholds[j]) & (
            articles_df["bias"] <= bias_thresholds[j+1])].index
    elif j == 1 :
        bias_indexes = articles_df[(articles_df["bias"] > bias_thresholds[j]) & (
            articles_df["bias"] <= bias_thresholds[j+1])].index
    elif j ==2:
        bias_indexes = articles_df[(articles_df["bias"] > bias_thresholds[j]) & (
            articles_df["bias"] < bias_thresholds[j+1])].index
    elif j == 3:
        bias_indexes = articles_df[(articles_df["bias"] >= bias_thresholds[j]) & (
            articles_df["bias"] < bias_thresholds[j+1])].index
    else:
        print("sdf")
    count += len(bias_indexes)
count

In [ ]:
count = 0 

for k in range(len(reliability_thresholds)-1):
    if k != (len(reliability_thresholds)-1):
        if k == 3:
            reliability_indexes = articles_df[(articles_df["reliability"] >= reliability_thresholds[k]) & (
                articles_df["reliability"] <= reliability_thresholds[k+1])].index
        else:
            reliability_indexes = articles_df[(articles_df["reliability"] >= reliability_thresholds[k]) & (
                articles_df["reliability"] < reliability_thresholds[k+1])].index
    else:
        reliability_indexes = articles_df.index
    count += len(reliability_indexes)
count

In [ ]:
len(all_posts_info)

In [ ]:
def get_raw_plt():
    import matplotlib.pyplot as plt
    plt.rcParams.update(matplotlib.rcParamsDefault)
    plt.rcParams.update({
        "font.family": 'sans-serif',  # use serif/main font for text elements
        "font.size": "25",
        # "font.weight": "bold",

    })
    return plt


counts = np.zeros((6, 5))

target_goals = ["normalized_number_of_posts", "number_of_posts", "number_of_articles", "number_of_reactions",
                "normalized_number_of_reactions", "likeCount", "shareCount", "commentCount", "deep_all"]
target_goals = ["number_of_articles", "number_of_posts", "normalized_number_of_posts", "number_of_reactions",
                "normalized_number_of_reactions"]

for target_goal in target_goals:
    for j in tqdm(range(len(bias_thresholds))):
        if j == len(bias_thresholds)-1:
            bias_indexes = articles_df.index
        elif j == 0 or j == 4:
            bias_indexes = articles_df[(articles_df["bias"] >= bias_thresholds[j]) & (
                articles_df["bias"] <= bias_thresholds[j+1])].index
        elif j == 1 :
            bias_indexes = articles_df[(articles_df["bias"] > bias_thresholds[j]) & (
                articles_df["bias"] <= bias_thresholds[j+1])].index
        elif j ==2:
            bias_indexes = articles_df[(articles_df["bias"] > bias_thresholds[j]) & (
                articles_df["bias"] < bias_thresholds[j+1])].index
        elif j == 3:
            bias_indexes = articles_df[(articles_df["bias"] >= bias_thresholds[j]) & (
                articles_df["bias"] < bias_thresholds[j+1])].index
            
        for k in range(len(reliability_thresholds)):
            if k != (len(reliability_thresholds)-1):
                if k == 3:
                    reliability_indexes = articles_df[(articles_df["reliability"] >= reliability_thresholds[k]) & (
                        articles_df["reliability"] <= reliability_thresholds[k+1])].index
                else:
                    reliability_indexes = articles_df[(articles_df["reliability"] >= reliability_thresholds[k]) & (
                        articles_df["reliability"] < reliability_thresholds[k+1])].index
            else:
                reliability_indexes = articles_df.index
             
            if len (reliability_indexes) == 0 :
                print("reliability_indexes is empty")
                
            indexes = list(set(bias_indexes).intersection(
                reliability_indexes))
            index_urls = articles_df.loc[indexes]["canonical_url"].values
            if target_goal == "number_of_articles":
                counts[j][k] = len(indexes)
            elif target_goal == "number_of_posts":
                cntt = 0
                for index in index_urls:
                    cntt += len(all_posts_info[index])
                counts[j][k] = cntt
            elif target_goal == "normalized_number_of_posts":
                cntt = 0
                for index in index_urls:
                    cntt += len(all_posts_info[index])
                counts[j][k] = cntt/len(indexes)
            elif target_goal == "number_of_reactions":
                cntt = 0
                for index in index_urls:
                    for post in all_posts_info[index]:
                        cntt += max(post["history"]["all_interactions"])
                counts[j][k] = cntt
            elif target_goal == "normalized_number_of_reactions":
                cntt = 0
                cntt2 = 0
                for index in index_urls:
                    for post in all_posts_info[index]:
                        cntt += max(post["history"]["all_interactions"])
                        cntt2 += 1
                counts[j][k] = cntt/cntt2
            elif target_goal == "likeCount":
                cntt = 0
                cntt2 = 0
                for index in index_urls:
                    for post in all_posts_info[index]:
                        cntt += max(post["history"]["likeCount"])
                        cntt2 += 1
                counts[j][k] = cntt/cntt2
            elif target_goal == "commentCount":
                cntt = 0
                cntt2 = 0
                for index in index_urls:
                    for post in all_posts_info[index]:
                        cntt += max(post["history"]["commentCount"])
                        cntt2 += 1
                counts[j][k] = cntt/cntt2
            elif target_goal == "shareCount":
                cntt = 0
                cntt2 = 0
                for index in index_urls:
                    for post in all_posts_info[index]:
                        cntt += max(post["history"]["shareCount"])
                        cntt2 += 1
                counts[j][k] = cntt/cntt2
           
    def human_format(num):
        num = float('{:.3g}'.format(num))
        magnitude = 0
        while abs(num) >= 1000:
            magnitude += 1
            num /= 1000.0
        return '{}{}'.format('{:f}'.format(num).rstrip('0').rstrip('.'), ['', 'K', 'M', 'B', 'T'][magnitude])
    
    
    fig, ax = plt.subplots()
    mycmap = cm.get_cmap('coolwarm', 1000)  # Define the colormap for the heatmap.
    norm = matplotlib.colors.LogNorm(vmin=counts.min(), vmax=counts.max())  # Set logarithmic normalization for the colormap.

    im = ax.imshow(counts, cmap=mycmap, norm=norm)  # Display the heatmap using the defined colormap and normalization.

    # Configure axis ticks and labels.
    ax.set_xticks(np.arange(len(reliability_classes)))
    ax.set_yticks(np.arange(len(bias_classes)))
    ax.set_xticklabels(reliability_classes, fontsize=35)
    ax.set_yticklabels(bias_classes, fontsize=35)
    ax.invert_yaxis()

    # Rotate x-axis tick labels.
    plt.setp(ax.get_xticklabels(), rotation=45, ha="right", rotation_mode="anchor")


    # Add the count values as text on each heatmap cell.
    for i in range(len(bias_classes)):
        for j in range(len(reliability_classes)):

            if target_goal == "number_of_reactions":
                text = ax.text(j, i, "{}".format(human_format(int(counts[i, j]))),
                               ha="center", va="center", color="0.0", fontsize=24, fontweight="bold")
            else:
                if target_goal != "deep_all":
                    text = ax.text(j, i, int(counts[i, j]),
                                   ha="center", va="center", color="0.0", fontsize=24, fontweight="bold")
                else:
                    text = ax.text(j, i, "{:.2f}".format(counts[i, j]),
                                   ha="center", va="center", color="0.0", fontsize=24, fontweight="bold")


    # Add a colorbar to the plot.
    cbar = fig.colorbar(im, pad=0.01)
    for t in cbar.ax.get_yticklabels():
        t.set_fontsize(24)

    # Save and display the plot.
    plt_export(target_goal)
    plt.show()


In [ ]:
for row in articles_df.groupby('source').size().sort_values(ascending=False).iteritems():
    print(row)

In [ ]:
moniker_names = ["The New York Times", 'Fox News (website)', 'CNN (website)', 'New York Post', 'NPR', 'Reuters']

In [ ]:
for moniker in moniker_names:
    moniker_df = articles_df[articles_df['source'] == moniker]
    print(moniker)
    
    cnt = 0
    for url in moniker_df["canonical_url"].values:
        cnt += len(all_posts_info[url])
    print("number of articles : ", moniker_df.shape[0])
    print("number of posts : ", cnt)
    print("number of reactions : ", sum([max(post["history"]["all_interactions"])
          for url in moniker_df["canonical_url"].values  for post in all_posts_info[url]]))
    print("normalized number of posts : ", cnt/moniker_df.shape[0])
    print("normalized number of reactions : ", sum([max(post["history"]["all_interactions"])
          for url in moniker_df["canonical_url"].values  for post in all_posts_info[url]])/cnt)
    print("")


In [ ]:
bias_thresholds

In [ ]:

# Function to get all the posts of a given bucket.
def get_all_posts_of_bucket(article_indexes, bucket_no, interaction_type, previous_bucket_results=None):
    post_results = []
    post_maxes = {}
    global all_posts_info
    for url in article_indexes:
        for post in all_posts_info[url]:
            if post["platformId"] not in excluded_posts:
                post_values = []
                post_history = post["history"]
                thres1 = buckets_thresholds[bucket_no]
                thres2 = buckets_thresholds[bucket_no+1]
                t = (post_history["date"] < thres2) & (
                    post_history["date"] >= thres1)
                post_values = post_history[interaction_type][t]

                if previous_bucket_results is not None:
                    previous_bucket_max = previous_bucket_results[post["platformId"]]
                else:
                    previous_bucket_max = 0
                if post_values.shape[0] == 0:
                    print("no values for post ", post["platformId"])
                assert post_values.shape[0] > 0
                post_maxes[post["platformId"]] = np.max(post_values)
                post_results.append(
                    (post_maxes[post["platformId"]]-previous_bucket_max)*100/np.max(post_history["all_interactions"]))
    return np.array(post_results), post_maxes


In [ ]:
post["history"].keys()

In [ ]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import cycler
import seaborn as sns
from mpl_toolkits.axes_grid1 import AxesGrid
from matplotlib.gridspec import GridSpec
import mpl_toolkits.axes_grid1.inset_locator as mpl_il
from scipy import stats
from scipy.stats import shapiro
from scipy.stats import kstest
from scipy.stats import ttest_1samp


In [ ]:
buckets_threshold_text = ['0 hours',
                          '1 hour and 17 minutes',
                          '5 hours and 16 minutes',
                          '17 hours and 28 minutes',
                          '22days', ]


In [ ]:
from matplotlib.gridspec import GridSpec
def get_raw_plt():
    import matplotlib.pyplot as plt
    plt.rcParams.update(matplotlib.rcParamsDefault)
    plt.rcParams.update({
        "font.family": 'sans-serif',  # use serif/main font for text elements
        "font.size": "25",
        # "font.weight": "bold",

    })
    return plt


for interaction_type in ['all_interactions','likeCount', 'shareCount', 'commentCount','angryCount','hahaCount']:
    print(interaction_type)
    temp_results = []

    bucket_results_averages = {}
    bucket_results_averages[interaction_type] = {}
    for i in range(len(buckets_thresholds)-1):
        bucket_results_averages[interaction_type][i] = np.zeros(
            (len(bias_thresholds), len(reliability_thresholds)))

    bucket_results_arrays = {}
    bucket_results_arrays[interaction_type] = {}
    for i in range(len(buckets_thresholds)-1):
        for j in range(len(bias_thresholds)):
            for k in range(len(reliability_thresholds)):
                bucket_results_arrays[interaction_type][(i, j, k)] = []

    for j in tqdm(range(len(bias_thresholds))):
        if j == len(bias_thresholds)-1:
            bias_indexes = articles_df.index
        elif j == 0 or j == 4:
            bias_indexes = articles_df[(articles_df["bias"] >= bias_thresholds[j]) & (
                articles_df["bias"] <= bias_thresholds[j+1])].index
        elif j == 1 :
            bias_indexes = articles_df[(articles_df["bias"] > bias_thresholds[j]) & (
                articles_df["bias"] <= bias_thresholds[j+1])].index
        elif j ==2:
            bias_indexes = articles_df[(articles_df["bias"] > bias_thresholds[j]) & (
                articles_df["bias"] < bias_thresholds[j+1])].index
        elif j == 3:
            bias_indexes = articles_df[(articles_df["bias"] >= bias_thresholds[j]) & (
                articles_df["bias"] < bias_thresholds[j+1])].index
            
        for k in range(len(reliability_thresholds)):
            if k != (len(reliability_thresholds)-1):
                if k == 3:
                    reliability_indexes = articles_df[(articles_df["reliability"] >= reliability_thresholds[k]) & (
                        articles_df["reliability"] <= reliability_thresholds[k+1])].index
                else:
                    reliability_indexes = articles_df[(articles_df["reliability"] >= reliability_thresholds[k]) & (
                        articles_df["reliability"] < reliability_thresholds[k+1])].index
            else:
                reliability_indexes = articles_df.index
                
            indexes = list(set(bias_indexes).intersection(
                reliability_indexes))
            
            for i in range(0, len(buckets_thresholds)-1):
                if i > 0:
                    previous_bucket_maxes = post_maxes
                else:
                    previous_bucket_maxes = None
                urls = articles_df.loc[indexes]["canonical_url"].values
                bucket_results, post_maxes = get_all_posts_of_bucket(
                    urls, i, interaction_type, previous_bucket_maxes)
                
                bucket_results_arrays[interaction_type][(
                    i, j, k)] = bucket_results

    for i in range(len(buckets_thresholds)-1):
        for j in range(len(bias_thresholds)):
            for k in range(len(reliability_thresholds)):
                bucket_results_averages[interaction_type][i][j, k] = np.mean(
                    (bucket_results_arrays[interaction_type][(i, j, k)]))
    plt = get_raw_plt()

    def see(data): return np.std(data, ddof=1) / np.sqrt(np.size(data))
    

    plt = get_raw_plt()
    bucket_results = []
    bucket_text = []


    for time_bucket_index in range(BUCKETS_NO):
        text = np.empty((6, 5), dtype=np.object_)
        result = bucket_results_averages[interaction_type][time_bucket_index]
        for bias_index in range(len(bias_thresholds)):
            for reliability_index in range(len(reliability_thresholds)):

                text[bias_index, reliability_index] = ""
                std_error = see(bucket_results_arrays[interaction_type][(
                    time_bucket_index, bias_index, reliability_index)])


                thresh_old = 4.0
                if bucket_results_averages[interaction_type][time_bucket_index][bias_index][reliability_index] != 0:
                    if std_error*100/bucket_results_averages[interaction_type][time_bucket_index][bias_index][reliability_index] < thresh_old:
                        text[bias_index, reliability_index] += "\u2605"
                if time_bucket_index > 0:
                    a = bucket_results_arrays[interaction_type][(
                        time_bucket_index, bias_index, reliability_index)]
                    b = bucket_results_arrays[interaction_type][(
                        time_bucket_index-1, bias_index, reliability_index)]
                    if stats.ttest_rel(a, b)[1] < 0.05:
                        if bucket_results_averages[interaction_type][time_bucket_index][bias_index, reliability_index] > bucket_results_averages[interaction_type][time_bucket_index-1][bias_index, reliability_index]:
                            text[bias_index, reliability_index] += "\u25B2"
                        else:
                            text[bias_index, reliability_index] += "\u25BC"


        bucket_results.append(result)
        bucket_text.append(text)

    plt.rcParams["figure.figsize"] = (20, 10.3)

    fig = plt.figure(constrained_layout=False)

    gs = GridSpec(1, BUCKETS_NO, figure=fig)

    ax_first_row = fig.add_subplot(gs[0, :])

    ax_first_row.arrow(0, .67, 5, 0, head_width=0.035,
                    width=0.005, facecolor='darkgreen', color=None)
    ax_first_row.text(2.5, 0.71, "Time", ha="center",
                    va="center",  size=24, color="darkgreen")

    x = [1.07, 2.52, 3.97]
    l = [-0.055, -0.045, -0.02]
    for i in range(len(x)):
        ax_first_row.arrow(x[i], .67, 0, l[i], facecolor='darkgreen',
                        color=None, width=0.005, head_width=0.03)
    '''

    ax_first_row.scatter([0],[-2])
    '''
    # ax_first_row.scatter([0,1],[1,1])
    ax_first_row.scatter([0, 1], [0, 0], color="white")
    ax_first_row.scatter([0, 1], [0, 0], color="white")

    for spine in ["left", "top", "right", "bottom"]:
        _ = ax_first_row.spines[spine].set_visible(False)
    _ = ax_first_row.set_xticks([])
    _ = ax_first_row.set_yticks([])

    axes = []

    for i in range(BUCKETS_NO):
        axes.append(fig.add_subplot(gs[0, i]))

    vim = 100
    vax = 0
    for t in range(BUCKETS_NO):
        if np.min(bucket_results[t]) < vim:
            vim = np.min(bucket_results[t])
        if np.max(bucket_results[t]) > vax:
            vax = np.max(bucket_results[t])
    #if vim < 1:
    #    vim = 1

    cs = []
    cs2 = []
    if interaction_type == "all_interactions":
        cs = [17, 20, 28.25, 29.5, 32]
        for i in np.arange(22.25, 28.25, 0.5):
            cs.append(i)

        cs.append(25.5)
        cs.sort()

        cs.sort()
    elif interaction_type == "likeCount":
        vax1 = 18
        vim1 = 5
        for i in range(vim1, vax1, 1):
            cs.append(i)
        for i in np.arange(7, 12.5, .5):
            cs.append(i)

        cs.append(9.25)
        cs.append(9.75)
        cs = sorted(set(cs))
    elif interaction_type == "shareCount":
        vax1 = 10
        vim1 = 3
        for i in range(vim1, vax1, 1):
            cs.append(i)
        for i in np.arange(2, 5.5, .5):
            cs.append(i)
        cs.append(4.25)
        cs = sorted(set(cs))
    elif interaction_type == "commentCount":
        vax1 = 7
        vim1 = 2
        for i in range(vim1, vax1, 1):
            cs.append(i)
        for i in np.arange(3, 6.5, .5):
            cs.append(i)
        cs.append(4.25)
        cs = sorted(set(cs))
    elif interaction_type == "angryCount":
        cs = [1, 1.5, 2, 2.5, 3, 3.5, 4, 4.5]
        cs = sorted(set(cs))

    elif interaction_type == "hahaCount":
        cs = [0.05, 1, 1.5, 2, 2.5 ]
        cs = sorted(set(cs))
    elif interaction_type == "loveCount":
        cs = [0, 0.25, 0.5, .75, 1, 1.5, 2, 2.5]
        cs = sorted(set(cs))

    cs.append(vax)
    cs.insert(0, vim)
    for elem in cs:
        if elem > vax or elem < vim:
            print(elem)
    mycmap = cm.get_cmap('coolwarm', 1000)

    bounds = np.array(cs)
    norm = matplotlib.colors.BoundaryNorm(
        boundaries=bounds, ncolors=1000, extend='both')

    for time_step in range(len(bucket_results)):
        ax = axes[time_step]
        im = ax.imshow(bucket_results[time_step], cmap=mycmap, norm=norm)

        ax.set_xticks(np.arange(len(reliability_classes)))
        ax.set_xticklabels(reliability_classes)
        if time_step == 0:
            ax.set_yticks(np.arange(len(bias_classes)))
            ax.set_yticklabels(bias_classes)
        else:
            ax.set_yticks([])
        ax.invert_yaxis()



        if time_step != 0:
            ax.text(-0.1, .5, buckets_threshold_text[time_step], bbox=dict(facecolor='green', alpha=0.5),
                    horizontalalignment='center', verticalalignment='center', rotation="vertical", transform=ax.transAxes, size=22)

        plt.setp(ax.get_xticklabels(), rotation=45, ha="right",
                 rotation_mode="anchor")
        for i in range(len(bias_classes)):
            for j in range(len(reliability_classes)):
                text = ax.text(j, i, bucket_text[time_step][i][j],
                               ha="center", va="center", color="0.0", fontsize=16)

    fig.tight_layout()
    cax = fig.add_axes([axes[-1].get_position().x1+0.01,
                       ax.get_position().y0-.4, 0.02, axes[-1].get_position().height+.3])
    cbar = fig.colorbar(im, cax=cax)

    for t in cbar.ax.get_yticklabels():
        t.set_fontsize(20)
    plt.subplots_adjust(wspace=.2, hspace=0.8, top=.62)


    plt_export(interaction_type)
    plt.show()


In [ ]:
moniker_names

In [ ]:
from matplotlib.gridspec import GridSpec
def get_raw_plt():
    import matplotlib.pyplot as plt
    plt.rcParams.update(matplotlib.rcParamsDefault)
    plt.rcParams.update({
        "font.family": 'sans-serif',  # use serif/main font for text elements
        "font.size": "25",
        # "font.weight": "bold",

    })
    return plt

for moniker in moniker_names:
    print(moniker)
    moniker_indexes = articles_df[articles_df['source'] == moniker].index
    for interaction_type in ['all_interactions']:
        print(interaction_type)
        temp_results = []

        bucket_results_averages = {}
        bucket_results_averages[interaction_type] = {}
        for i in range(len(buckets_thresholds)-1):
            bucket_results_averages[interaction_type][i] = np.zeros(
                (len(bias_thresholds), len(reliability_thresholds)))

        bucket_results_arrays = {}
        bucket_results_arrays[interaction_type] = {}
        for i in range(len(buckets_thresholds)-1):
            for j in range(len(bias_thresholds)):
                for k in range(len(reliability_thresholds)):
                    bucket_results_arrays[interaction_type][(i, j, k)] = []

        for j in tqdm(range(len(bias_thresholds))):
            if j == len(bias_thresholds)-1:
                bias_indexes = articles_df.index
            elif j == 0 or j == 4:
                bias_indexes = articles_df[(articles_df["bias"] >= bias_thresholds[j]) & (
                    articles_df["bias"] <= bias_thresholds[j+1])].index
            elif j == 1 :
                bias_indexes = articles_df[(articles_df["bias"] > bias_thresholds[j]) & (
                    articles_df["bias"] <= bias_thresholds[j+1])].index
            elif j ==2:
                bias_indexes = articles_df[(articles_df["bias"] > bias_thresholds[j]) & (
                    articles_df["bias"] < bias_thresholds[j+1])].index
            elif j == 3:
                bias_indexes = articles_df[(articles_df["bias"] >= bias_thresholds[j]) & (
                    articles_df["bias"] < bias_thresholds[j+1])].index
                
            for k in range(len(reliability_thresholds)):
                if k != (len(reliability_thresholds)-1):
                    if k == 3:
                        reliability_indexes = articles_df[(articles_df["reliability"] >= reliability_thresholds[k]) & (
                            articles_df["reliability"] <= reliability_thresholds[k+1])].index
                    else:
                        reliability_indexes = articles_df[(articles_df["reliability"] >= reliability_thresholds[k]) & (
                            articles_df["reliability"] < reliability_thresholds[k+1])].index
                else:
                    reliability_indexes = articles_df.index
                    
                indexes = list(set(bias_indexes).intersection(
                    reliability_indexes).intersection(moniker_indexes))
                
                for i in range(0, len(buckets_thresholds)-1):
                    if i > 0:
                        previous_bucket_maxes = post_maxes
                    else:
                        previous_bucket_maxes = None
                    urls = articles_df.loc[indexes]["canonical_url"].values
                    bucket_results, post_maxes = get_all_posts_of_bucket(
                        urls, i, interaction_type, previous_bucket_maxes)
                    
                    bucket_results_arrays[interaction_type][(
                        i, j, k)] = bucket_results

        for i in range(len(buckets_thresholds)-1):
            for j in range(len(bias_thresholds)):
                for k in range(len(reliability_thresholds)):
                    if len(bucket_results_arrays[interaction_type][(i, j, k)]) > 0:
                        bucket_results_averages[interaction_type][i][j, k] = np.mean((bucket_results_arrays[interaction_type][(i, j, k)]))
                    else:
                        bucket_results_averages[interaction_type][i][j, k] = np.nan
        plt = get_raw_plt()

        def see(data):
            if len(data) <= 1:
                return np.nan
            return np.std(data, ddof=1) / np.sqrt(np.size(data))

        plt = get_raw_plt()
        bucket_results = []
        bucket_text = []



        for time_bucket_index in range(BUCKETS_NO):
            text = np.empty((6, 5), dtype=np.object_)
            result = bucket_results_averages[interaction_type][time_bucket_index]
            for bias_index in range(len(bias_thresholds)):
                for reliability_index in range(len(reliability_thresholds)):

                    text[bias_index, reliability_index] = ""
                    std_error = see(bucket_results_arrays[interaction_type][(
                        time_bucket_index, bias_index, reliability_index)])

                    if interaction_type == 'all_interactions':
                        thresh_old = 4.0
                    else:
                        thresh_old = 4.0
                    if bucket_results_averages[interaction_type][time_bucket_index][bias_index][reliability_index] != 0:
                        if std_error*100/bucket_results_averages[interaction_type][time_bucket_index][bias_index][reliability_index] < thresh_old:
                            text[bias_index, reliability_index] += "\u2605"
                    if time_bucket_index > 0:
                        a = bucket_results_arrays[interaction_type][(
                            time_bucket_index, bias_index, reliability_index)]
                        b = bucket_results_arrays[interaction_type][(
                            time_bucket_index-1, bias_index, reliability_index)]
                        if len(a) > 1 and len(b) > 1:
                            if stats.ttest_rel(a, b)[1] < 0.05:
                                if bucket_results_averages[interaction_type][time_bucket_index][bias_index, reliability_index] > bucket_results_averages[interaction_type][time_bucket_index-1][bias_index, reliability_index]:
                                    text[bias_index, reliability_index] += "\u25B2"
                                else:
                                    text[bias_index, reliability_index] += "\u25BC"


            bucket_results.append(result)
            bucket_text.append(text)

        plt.rcParams["figure.figsize"] = (20, 10.3)

        fig = plt.figure(constrained_layout=False)

        gs = GridSpec(1, BUCKETS_NO, figure=fig)

        ax_first_row = fig.add_subplot(gs[0, :])

        ax_first_row.arrow(0, .67, 5, 0, head_width=0.035,
                        width=0.005, facecolor='darkgreen', color=None)
        ax_first_row.text(2.5, 0.71, "Time", ha="center",
                        va="center",  size=24, color="darkgreen")

        x = [1.07, 2.52, 3.97]
        l = [-0.055, -0.045, -0.02]
        for i in range(len(x)):
            ax_first_row.arrow(x[i], .67, 0, l[i], facecolor='darkgreen',
                            color=None, width=0.005, head_width=0.03)
        '''

        ax_first_row.scatter([0],[-2])
        '''
        ax_first_row.scatter([0, 1], [0, 0], color="white")
        ax_first_row.scatter([0, 1], [0, 0], color="white")

        for spine in ["left", "top", "right", "bottom"]:
            _ = ax_first_row.spines[spine].set_visible(False)
        _ = ax_first_row.set_xticks([])
        _ = ax_first_row.set_yticks([])

        axes = []

        for i in range(BUCKETS_NO):
            axes.append(fig.add_subplot(gs[0, i]))

        vim = 100
        vax = 0
        for t in range(BUCKETS_NO):
            if np.nanmin(bucket_results[t]) < vim:
                vim = np.nanmin(bucket_results[t])
            if np.nanmax(bucket_results[t]) > vax:
                vax = np.nanmax(bucket_results[t])


        cs = []
        cs2 = []
        if interaction_type == "all_interactions":
            cs = [ 20, 28.25, 29.5]
            for i in np.arange(22.25, 28.25, 0.5):
                cs.append(i)

            cs.append(25.5)
            cs.sort()

            cs.sort()
        elif interaction_type == "likeCount":
            vax1 = 18
            vim1 = 5
            for i in range(vim1, vax1, 1):
                cs.append(i)
            for i in np.arange(7, 12.5, .5):
                cs.append(i)

            cs.append(9.25)
            cs.append(9.75)
            cs = sorted(set(cs))
        elif interaction_type == "shareCount":
            vax1 = 10
            vim1 = 3
            for i in range(vim1, vax1, 1):
                cs.append(i)
            for i in np.arange(2, 5.5, .5):
                cs.append(i)
            cs.append(4.25)
            cs = sorted(set(cs))
        elif interaction_type == "commentCount":
            vax1 = 7
            vim1 = 2
            for i in range(vim1, vax1, 1):
                cs.append(i)
            for i in np.arange(3, 6.5, .5):
                cs.append(i)
            cs.append(4.25)
            cs = sorted(set(cs))
        elif interaction_type == "angryCount":
            cs = [1, 1.5, 2, 2.5, 3, 3.5, 4, 4.5]
            cs = sorted(set(cs))

        elif interaction_type == "hahaCount":
            cs = [0.05, 1, 1.5, 2, 2.5 ]
            cs = sorted(set(cs))
        elif interaction_type == "loveCount":
            cs = [0, 0.25, 0.5, .75, 1, 1.5, 2, 2.5]
            cs = sorted(set(cs))

        cs.append(vax)
        cs.insert(0, vim)
        for elem in cs:
            if elem > vax or elem < vim:
                print(elem)
        mycmap = cm.get_cmap('coolwarm', 1000)

        bounds = np.array(cs)
        norm = matplotlib.colors.BoundaryNorm(
            boundaries=bounds, ncolors=1000, extend='both')

        for time_step in range(len(bucket_results)):
            ax = axes[time_step]
            im = ax.imshow(bucket_results[time_step], cmap=mycmap, norm=norm)

            ax.set_xticks(np.arange(len(reliability_classes)))
            ax.set_xticklabels(reliability_classes)
            if time_step == 0:
                ax.set_yticks(np.arange(len(bias_classes)))
                ax.set_yticklabels(bias_classes)
            else:
                ax.set_yticks([])
            ax.invert_yaxis()


            if time_step != 0:
                ax.text(-0.1, .5, buckets_threshold_text[time_step], bbox=dict(facecolor='green', alpha=0.5),
                        horizontalalignment='center', verticalalignment='center', rotation="vertical", transform=ax.transAxes, size=22)

            plt.setp(ax.get_xticklabels(), rotation=45, ha="right",
                    rotation_mode="anchor")
            for i in range(len(bias_classes)):
                for j in range(len(reliability_classes)):
                    text = ax.text(j, i, bucket_text[time_step][i][j],
                                ha="center", va="center", color="0.0", fontsize=16)

        fig.tight_layout()
        cax = fig.add_axes([axes[-1].get_position().x1+0.01,
                        ax.get_position().y0-.4, 0.02, axes[-1].get_position().height+.3])
        cbar = fig.colorbar(im, cax=cax)

        for t in cbar.ax.get_yticklabels():
            t.set_fontsize(20)
        plt.subplots_adjust(wspace=.2, hspace=0.8, top=.62)

        plt_export(moniker+"_"+interaction_type)
        plt.show()
